In [14]:
import os
import math

import cv2
import numpy as np
import paddle

from paddleseg import utils
from paddleseg.core import infer
from paddleseg.utils import logger, progbar, visualize, get_image_list
from paddleseg.cvlibs import manager
from paddleseg.transforms import Compose

In [22]:
model = manager.MODELS.components_dict['SegFormer_B5'](num_classes=20)
model_path = './best_model/model.pdparams'

In [23]:
transforms = Compose([manager.TRANSFORMS['Normalize']()])
transforms

In [24]:
image_list, image_dir = get_image_list('../datasets/test/image/')

In [25]:
image_list, image_dir

(['../datasets/test/image/08909_00.jpg', '../datasets/test/image/m_00002.jpg'],
 '../datasets/test/image/')

In [39]:
import pickle
with open('../colormap.pkl', 'rb') as file:
    color_map = pickle.load(file)

In [41]:
# 高层 Paddleseg API
from paddleseg.core import predict
predict(model, 
        model_path=model_path, 
        transforms=transforms, 
        image_list=image_list, 
        image_dir=image_dir, 
        save_dir='output',
        custom_color=color_map)

2022-03-08 14:51:54 [INFO]	Loading pretrained model from ./best_model/model.pdparams
2022-03-08 14:51:54 [INFO]	There are 1067/1067 variables loaded into SegFormer.
2022-03-08 14:51:54 [INFO]	Start to predict...


2/2 [==============================] - 0s 225ms/step


In [28]:
# 仅做 inference
utils.utils.load_entire_model(model, model_path)
model.eval()

2022-03-08 14:48:48 [INFO]	Loading pretrained model from ./best_model/model.pdparams
2022-03-08 14:48:49 [INFO]	There are 1067/1067 variables loaded into SegFormer.


In [30]:
img_lists = [image_list]
print(img_lists)

[['../datasets/test/image/08909_00.jpg', '../datasets/test/image/m_00002.jpg']]


In [34]:
logger.info("start to predict...")
progbar_pred = progbar.Progbar(target=len(img_lists[0]), verbose=1)
print(progbar_pred)

2022-03-08 14:49:41 [INFO]	start to predict...
